In [1]:
import os
import pygame
import random
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
os.environ["PYGAME_HIDE_SUPPORT_PROMPT"] = "1"

# Initialisation de Pygame
pygame.init()

# Paramètres
GRID_SIZE = 200
WINDOW_SIZE = 800
TAXI_RADIUS = 5
FPS = 60
NUM_TAXIS = 3
NUM_NEW_TASKS_MIN = 4
NUM_NEW_TASKS_MAX = 4 # Si <= à NUM_NEW_TASKS_MIN le nombre de tâches générées sera de NUM_NEW_TASKS_MIN
T = 5

# Échelle pour convertir les coordonnées
scale = WINDOW_SIZE / GRID_SIZE

# Couleurs
WHITE = (247, 247, 255)
GREY = (50, 60, 68)
YELLOW = (255, 217, 25)
RED = (199, 0, 57)
GREEN = (76, 173, 139)
BLUE = (57, 88, 146)

# Police pour l'affichage du texte
FONT = pygame.font.Font(None, 24)

In [3]:
def random_task_ordering(taxi_pos, tasks):
    ordered_tasks = tasks.copy()
    random.shuffle(ordered_tasks)
    return ordered_tasks

def greedy_task_ordering(taxi_pos, tasks):
    ordered_tasks = []
    current_position = taxi_pos
    remaining_tasks = tasks.copy()
    
    while remaining_tasks:
        closest_task = min(remaining_tasks, key=lambda task: ((task.start[0] - current_position[0])**2 + (task.start[1] - current_position[1])**2)**0.5)
        ordered_tasks.append(closest_task)
        current_position = closest_task.end
        remaining_tasks.remove(closest_task)
    
    print("Coût des tâches pour le taxi: ", sum([((task.start[0] - task.end[0])**2 + (task.start[1] - task.end[1])**2)**0.5 for task in ordered_tasks]))
    return ordered_tasks

def optimal_task_ordering(taxi_pos, tasks):
    best_order = None
    min_distance = float('inf')

    for perm in itertools.permutations(tasks):
        total_distance = 0
        current_position = taxi_pos
        
        for task in perm:
            total_distance += ((task.start[0] - current_position[0])**2 + (task.start[1] - current_position[1])**2) ** 0.5
            total_distance += ((task.end[0] - task.start[0])**2 + (task.end[1] - task.start[1])**2) ** 0.5
            current_position = task.end
        
        if total_distance < min_distance:
            min_distance = total_distance
            best_order = perm
    
    return list(best_order) if best_order else []

In [4]:
# Fonction pour dessiner du texte
def draw_text(screen, text, x, y, color):
    text_surface = FONT.render(text, True, color)
    screen.blit(text_surface, (x, y))

# Class des Taxis
class Taxi:

    def __init__(self, position, name):
        self.name = name
        self.position = position
        self.all_tasks = [] # liste de toutes les tâches disponibles
        self.path = [] # liste des tâches à effectuer
        self.current_task = None
        self.destination = None
        self.task_started = False
        self.speed = 1

    def update(self):
        self.update_task()
        if self.destination:
            self.move(self.destination)

    def draw(self, screen):
        # Convertir la position à l'échelle de l'écran
        pos_scaled = (int(self.position[0] * scale), int(self.position[1] * scale))
        pygame.draw.circle(screen, YELLOW, pos_scaled, TAXI_RADIUS)

        # Afficher la destination au-dessus du taxi
        if self.destination:
            dest_scaled = (
                int(self.destination[0] * scale),
                int(self.destination[1] * scale),
            )
            text = f"Dest: {dest_scaled[0]}, {dest_scaled[1]}"
            draw_text(screen, text, pos_scaled[0], pos_scaled[1] - 15, WHITE)

    def move(self, destination):
        dx = destination[0] - self.position[0]
        dy = destination[1] - self.position[1]
        distance = (dx**2 + dy**2) ** 0.5
        if distance < self.speed:   # Si la distance est inférieure à la vitesse, on atteint la destination
            self.position = destination
            self.destination = None
        else:
            self.position = (
                    self.position[0] + (dx / distance) * self.speed,
                    self.position[1] + (dy / distance) * self.speed,
                )
            
    def check_task(self, tasks):
        tasks_changed = False

        for task in tasks:
            if task not in self.all_tasks:
                self.all_tasks.append(task)
                tasks_changed = True

        for task in self.all_tasks: # si la tâche n'est plus disponible ou déjà prise
            if task not in tasks or  task.taken:
                self.all_tasks.remove(task)
                tasks_changed = True

        if tasks_changed:
            self.path = self.all_tasks.copy()
            print("Pour le taxi ", self.name, " : \n")
            self.path = greedy_task_ordering(self.position, self.all_tasks)
            
            



    def update_task(self):
        
        if self.current_task: # si le taxi a une tâche en cours
            if self.position != self.current_task.end: # si le taxi n'est pas encore arrivé à la fin
                return
            
            self.current_task.complete()
            self.current_task = None
            self.destination = None

        if self.path: # si le taxi a une liste de tâches
            if self.position == self.path[0].start and not self.path[0].taken: # si le taxi est à la position de départ de la tâche et que la tâche n'est pas prise
                self.current_task = self.path.pop(0) # on remove la tâche de sa liste
                self.current_task.take()
                self.destination = self.current_task.end # on met la destination à la fin de la tâche
                return
            
            self.destination = self.path[0].start # sinon on met la destination à la position de départ de la tâche
            return
        
        # Si le taxi n'a plus de tâche à effectuer
        x_noise = random.uniform(-0.5, 0.5)
        y_noise = random.uniform(-0.5, 0.5)
        self.destination = (GRID_SIZE // 2 + x_noise, GRID_SIZE // 2 +  y_noise)
        # self.destination = (self.position[0] + x_noise, self.position[1] + y_noise)

In [5]:
# Class des Tâches
class Task:
    def __init__(self, start, end, name):
        self.name = name
        self.start = start
        self.end = end
        self.distance = ((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2) ** 0.5
        self.taken = False
        self.completed = False

    def draw(self, screen):
        color = WHITE if not self.taken else BLUE
        start_scaled = (int(self.start[0] * scale), int(self.start[1] * scale))
        end_scaled = (int(self.end[0] * scale), int(self.end[1] * scale))
        pygame.draw.circle(screen, GREEN, start_scaled, 3)
        pygame.draw.circle(screen, RED, end_scaled, 3)
        pygame.draw.line(screen, color, start_scaled, end_scaled, 2)


    def take(self):
        self.taken = True

    def complete(self):
        self.completed = True

In [6]:
def main():
    # Initialisation de la fenêtre
    screen = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
    pygame.display.set_caption("Simulation de taxis")
    

    # Génération aléatoire de taxis et de tâches
    taxis = [
        Taxi((random.randint(0, GRID_SIZE), random.randint(0, GRID_SIZE)), f"T{i+1}")
        for i in range(NUM_TAXIS)
    ]
    tasks = [
        Task(
            (random.randint(0, GRID_SIZE), random.randint(0, GRID_SIZE)),
            (random.randint(0, GRID_SIZE), random.randint(0, GRID_SIZE)),
            f"t{i+1}"
        )
        for i in range(NUM_NEW_TASKS_MIN)
    ]

    
    # Boucle principale
    running = True
    clock = pygame.time.Clock()
    start = time.time()

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Dessin de l'environnement
        screen.fill(GREY)

        if time.time() - start > T:
            start = time.time()
            num_new_tasks = random.randint(NUM_NEW_TASKS_MIN, NUM_NEW_TASKS_MAX)
            for _ in range(num_new_tasks):
                tasks.append(
                    Task(
                        (random.randint(0, GRID_SIZE), random.randint(0, GRID_SIZE)),
                        (random.randint(0, GRID_SIZE), random.randint(0, GRID_SIZE)),
                        f"t{len(tasks)+1}"
                    )
                )

        for t in taxis:
            available_tasks = [task for task in tasks if not task.taken]
            # pour calculer le temps de résolution
            t.check_task(available_tasks)
           

        # Déplacement des taxis
        for taxi in taxis:
            taxi.update()
            taxi.draw(screen)

        # Dessin des tâches
        for task in tasks:
            if task.completed:
                tasks.remove(task)
                continue
            
            task.draw(screen)
            
        
        
        pygame.display.flip()
        clock.tick(FPS)
        
        
        
        

    pygame.quit()
    

In [7]:
video = main()
display(video)

Pour le taxi  T1  : 

Coût des tâches pour le taxi:  321.9125559024094
Pour le taxi  T2  : 

Coût des tâches pour le taxi:  321.9125559024094
Pour le taxi  T3  : 

Coût des tâches pour le taxi:  321.9125559024094
Pour le taxi  T1  : 

Coût des tâches pour le taxi:  232.1016434251386
Pour le taxi  T2  : 

Coût des tâches pour le taxi:  232.1016434251386
Pour le taxi  T3  : 

Coût des tâches pour le taxi:  232.1016434251386
Pour le taxi  T1  : 

Coût des tâches pour le taxi:  153.83926421264596
Pour le taxi  T2  : 

Coût des tâches pour le taxi:  153.83926421264596
Pour le taxi  T3  : 

Coût des tâches pour le taxi:  153.83926421264596
Pour le taxi  T1  : 

Coût des tâches pour le taxi:  121.82364302548172
Pour le taxi  T2  : 

Coût des tâches pour le taxi:  121.82364302548172
Pour le taxi  T3  : 

Coût des tâches pour le taxi:  121.82364302548172
Pour le taxi  T1  : 

Coût des tâches pour le taxi:  0
Pour le taxi  T2  : 

Coût des tâches pour le taxi:  0
Pour le taxi  T3  : 

Coût des t

None